In [8]:
import torch
import evaluate # For metrics
from datasets import load_dataset, DatasetDict, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import os
from pathlib import Path

In [19]:
# --- Configuration ---
MODEL_NAME = "openai/whisper-small" 
LANGUAGE = "english" # MusicCaps captions are in English
TASK = "transcribe" # Whisper needs a task, 'transcribe' works for captioning fine-tuning
OUTPUT_DIR = "./whisper-musiccaps-finetuned"
MAX_SAMPLES = 1000 # Limit dataset size for faster testing/debugging. Set to None to use all data.
NUM_TRAIN_EPOCHS = 3
LEARNING_RATE = 1e-5
PER_DEVICE_TRAIN_BATCH_SIZE = 8 # Adjust based on GPU memory
PER_DEVICE_EVAL_BATCH_SIZE = 8  # Adjust based on GPU memory
GRADIENT_ACCUMULATION_STEPS = 2 # Effective batch size = train_batch_size * num_gpus * grad_accum
FP16 = True

DATA_DIR = Path("../data")
CSV_PATH = DATA_DIR / "musiccaps-public.csv"
AUDIO_DIR = DATA_DIR / "audio" 
AUDIO_EXTENSION = ".wav"


In [20]:
print(CSV_PATH)
if not CSV_PATH.is_file():
    raise FileNotFoundError(f"Metadata CSV not found at: {CSV_PATH}")

try:
    raw_metadata_dataset = load_dataset("csv", data_files=str(CSV_PATH))
    print("CSV metadata loaded successfully.")
    print(f"Number of entries in CSV: {len(raw_metadata_dataset)}")
except Exception as e:
    print(f"Failed to load CSV from {CSV_PATH}: {e}")
    raise 

../data/musiccaps-public.csv


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 987.36it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/py/several-ducks-datahacks25/.venv/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 5521 examples [00:00, 121373.62 examples/s]

Failed to load CSV from ../data/musiccaps-public.csv: Loading a dataset cached in a LocalFileSystem is not supported.


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.